In [202]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [203]:
from collections import defaultdict
import requests
from io import BytesIO

import pandas as pd 
import jellyfish
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm as tqdm

In [204]:
def get_all_sources(links):
    dfs = []
    for link in links:
        if "http" in link:
            html_res = requests.get(link)
            if html_res.status_code != 200:
                print(f"Failed to retrieve {link}")
                continue
            else:
                df = pd.read_csv(BytesIO(html_res.content))
            
        else:
            df = pd.read_csv(link)
            
        dfs.append(df)
    
    return dfs

In [205]:
source_links = ["https://docs.google.com/spreadsheets/d/1eLrq1zUQYbETRE_KOChBWspo0EpiNrBC_IImSRUhjVQ/export?format=csv", 
                "bd_technologies.csv"]
                # "https://drive.google.com/file/d/1YzV-Z3tzVpBibFxhqP08ax-2S6NES0xs/?usp=sharing"]


dfs = get_all_sources(source_links)

In [206]:
dfs[0].head()

,technology_id,name,slug,url,description,category,category_slug,parent_category,parent_category_slug,jobs,companies,companies_found_last_week
0,8,ActiveCampaign,activecampaign,http://www.activecampaign.com,Recognized as the leader in the marketing and ...,Marketing Automation Platforms,marketing-automation,Marketing,marketing,9845,4728,43
1,12,Acuity Scheduling,acuity-scheduling,https://acuityscheduling.com,It is easy-to-use and user friendly scheduling...,Appointments And Scheduling,appointments-and-scheduling,Customer Management,customer-management,322,167,4
2,17,Adobe Illustrator,adobe-illustrator,https://www.adobe.com/ru/products/illustrator....,The industry-standard vector graphics app lets...,Graphic Design Software,graphic-design-software,Product And Design,product-and-design,435235,181373,1201
3,18,Adobe Photoshop,adobe-photoshop,https://www.adobe.com/,It is the best in the world of graphic design ...,Graphic Design Software,graphic-design-software,Product And Design,product-and-design,539058,214280,1397
4,36,AfterShip,aftership,https://www.aftership.com,AfterShip provides shipment tracking API for o...,Shipping And Fulfillment,shipping-and-fulfillment,E-commerce,e-commerce,77,44,0


In [207]:
dfs[1].head()

,product_name,description,seller_description,seller_website,main_category,software_product_id,overview,headquarters,categories
0,Stellar Toolkit for iPhone,Stellar Toolkit for iPhone is a comprehensive ...,"Established in 1993, Stellar® is a global lead...",https://www.stellarinfo.com/,Data Recovery Software,stellar-toolkit-for-iphone,Stellar Toolkit for iPhone is a comprehensive ...,"Gurgaon, Haryana","[""File Recovery""]"
1,OpenText Voltage SmartCipher,Simplify unstructured data security with persi...,OpenText software applications manage content ...,https://www.opentext.com/,Confidentiality Software,opentext-voltage-smartcipher,SmartCipher simplifies unstructured data secur...,"Waterloo, ON","[""Encryption""]"
2,IBM Maximo IT,Rapid expansion of information technology has ...,IBM offers a wide range of technology and cons...,https://www.ibm.com/,Service Desk Software,ibm-maximo-it,IBM Control Desk enables unified IT Service an...,"Armonk, NY","[""IT Service Management (ITSM) Tools"",""Service..."
3,Panvalet,CA Panvalet is a library management system tha...,Broadcom Inc. (NASDAQ: AVGO) is a global techn...,https://www.broadcom.com/,DevOps Software,panvalet,CA Panvalet is a library management system tha...,"San Jose, CA","[""Version Control Software""]"
4,OpenText ZENworks Patch Management,Micro Focus ZENworks Patch Management (formerl...,OpenText software applications manage content ...,https://www.opentext.com/,Vulnerability Management Software,opentext-zenworks-patch-management,Micro Focus ZENworks Patch Management (formerl...,"Waterloo, ON","[""Patch Management""]"


In [208]:
print(dfs[0].describe())
print(dfs[1].describe())

       technology_id          jobs      companies  companies_found_last_week
count   32197.000000  3.219700e+04   32197.000000               32197.000000
mean   100681.749231  3.872473e+03     790.560332                   5.626798
std     43498.661615  5.307573e+04    8162.288487                  59.607930
min         8.000000  0.000000e+00       0.000000                   0.000000
25%    109065.000000  0.000000e+00       0.000000                   0.000000
50%    120143.000000  8.000000e+00       4.000000                   0.000000
75%    128277.000000  1.210000e+02      50.000000                   0.000000
max    136401.000000  3.822115e+06  605860.000000                4433.000000
       product_name                                        description  \
count         75975                                              75975   
unique        75173                                              75539   
top           Atlas  It takes an image as input and classifies the ...   
freq       

In [209]:
def quality_checks(df):
    # data types
    data_types = df.dtypes
    print("Data types:\n")
    print(data_types)

    # missing values
    print(f"Missing values:\n")
    for col in df.columns:
        print(f"  {col}: {df[col].isna().sum()} / {len(df[col])}")

    # naive duplicate rows
    duplicate_rows = df[df.duplicated()]
    print("Duplicate rows:\n", duplicate_rows.shape[0])

    # check that all numeric columns have positive values
    for col in ["technology_id", "jobs", "companies", "companies_found_last_year"]:
        if col in df.columns:
            if (df[col] < 0).any():
                print(f"Negative values found in {col}")

In [210]:
for df in dfs:
    quality_checks(df)

Data types:

technology_id                 int64
name                         object
slug                         object
url                          object
description                  object
category                     object
category_slug                object
parent_category              object
parent_category_slug         object
jobs                          int64
companies                     int64
companies_found_last_week     int64
dtype: object
Missing values:

  technology_id: 0 / 32197
  name: 0 / 32197
  slug: 0 / 32197
  url: 24731 / 32197
  description: 23804 / 32197
  category: 0 / 32197
  category_slug: 0 / 32197
  parent_category: 0 / 32197
  parent_category_slug: 0 / 32197
  jobs: 0 / 32197
  companies: 0 / 32197
  companies_found_last_week: 0 / 32197
Duplicate rows:
 0
Data types:

product_name           object
description            object
seller_description     object
seller_website         object
main_category          object
software_product_id    object
overvie

In [211]:
# 1st DF: url and description have significant missing values
# 2nd DF: small missing values from seller_website and categories, large missing values for headquarters. software_product_id also reads as the slug from first dataset. categories should also be sanitized (to remove square brackets and quotes)

In [212]:
print(dfs[1][dfs[1]["seller_website"].isna()])

                  product_name  \
240    Data Quality Monitoring   
25128           IBM QRadar XDR   
25131               Statusbrew   
26483        Loyalty & Rewards   
52337              Titan Forms   
71853           TestResults.io   

                                             description  \
240    Datachecks is an open-source data monitoring t...   
25128  IBM QRadar® XDR is a cloud-native solution tha...   
25131  Statusbrew is a social media management platfo...   
26483  Loloyal is a Shopify app for loyalty programs,...   
52337  TITAN is a no code Digital Experience cloud pl...   
71853  Test user flows from beginning to end across d...   

                                      seller_description seller_website  \
240    Datachecks is an open-source data monitoring t...            NaN   
25128  IBM offers a wide range of technology and cons...            NaN   
25131  Statusbrew is a comprehensive social media man...            NaN   
26483  Key features of Loloyal: ● Point 

## Data Cleaning

1. Rename columns that represent the same information - name / product_name, url / seller_website
2. Missing values. For now add a placeholder term "UNKNOWN" (because an empty string might give an exact match in entity matching).
    ts_technologies:
      - "seller_website", 
      - "categories", 
      - "headquarters" 
    bd_technologies:
      - "url",
      - "description".
3. Remove square brackets and quotes from "categories"
4. Unify categories in bd_technologies

In [213]:
# rename same columns
dfs[1].rename(columns={"product_name": "name", "seller_website": "url", "main_category": "parent_category", "categories": "category"}, inplace=True)

# bd_technologies: first fill in the 3 missing values with empty string
for col in ["url", "category", "headquarters"]:
    dfs[1].loc[dfs[1][col].isna(), col] = "UNKNOWN"

# ts_technologies: do the same for this one
for col in ["url", "description"]:
    dfs[0].loc[dfs[0][col].isna(), col] = "UNKNOWN"


# remove square brackets and quotes from categories
dfs[1]["category"] = dfs[1]["category"].apply(lambda x: x.strip("[]\"").replace(",", " "))

print(dfs[0].isna().sum())
print(dfs[1].isna().sum())

technology_id                0
name                         0
slug                         0
url                          0
description                  0
category                     0
category_slug                0
parent_category              0
parent_category_slug         0
jobs                         0
companies                    0
companies_found_last_week    0
dtype: int64
name                   0
description            0
seller_description     0
url                    0
parent_category        0
software_product_id    0
overview               0
headquarters           0
category               0
dtype: int64


In [214]:
dfs[0]

,technology_id,name,slug,url,description,category,category_slug,parent_category,parent_category_slug,jobs,companies,companies_found_last_week
0,8,ActiveCampaign,activecampaign,http://www.activecampaign.com,Recognized as the leader in the marketing and ...,Marketing Automation Platforms,marketing-automation,Marketing,marketing,9845,4728,43
1,12,Acuity Scheduling,acuity-scheduling,https://acuityscheduling.com,It is easy-to-use and user friendly scheduling...,Appointments And Scheduling,appointments-and-scheduling,Customer Management,customer-management,322,167,4
2,17,Adobe Illustrator,adobe-illustrator,https://www.adobe.com/ru/products/illustrator....,The industry-standard vector graphics app lets...,Graphic Design Software,graphic-design-software,Product And Design,product-and-design,435235,181373,1201
3,18,Adobe Photoshop,adobe-photoshop,https://www.adobe.com/,It is the best in the world of graphic design ...,Graphic Design Software,graphic-design-software,Product And Design,product-and-design,539058,214280,1397
4,36,AfterShip,aftership,https://www.aftership.com,AfterShip provides shipment tracking API for o...,Shipping And Fulfillment,shipping-and-fulfillment,E-commerce,e-commerce,77,44,0
...,...,...,...,...,...,...,...,...,...,...,...,...
32192,136396,Sketchfab,sketchfab,UNKNOWN,"With a community of over one million creators,...",3D Modeling,3d-modeling,Product And Design,product-and-design,106,46,0
32193,136397,PMWeb,pmweb,UNKNOWN,"PMWeb is the World's #1 All-in-One, Enterprise...",Construction Management,construction-management,Productivity And Operations,productivity-and-operations,842,232,1
32194,136398,Murex,murex,UNKNOWN,"Transform IT infrastructure, meet regulatory r...",Investment And Finance Services,investment-and-finance-services,Finance And Accounting,finance-and-accounting,8087,1540,11
32195,136399,Calypso,calypso,UNKNOWN,Calypso is the leading provider of innovative ...,Investment And Finance Services,investment-and-finance-services,Finance And Accounting,finance-and-accounting,7410,2516,32


In [215]:
dfs[1]

,name,description,seller_description,url,parent_category,software_product_id,overview,headquarters,category
0,Stellar Toolkit for iPhone,Stellar Toolkit for iPhone is a comprehensive ...,"Established in 1993, Stellar® is a global lead...",https://www.stellarinfo.com/,Data Recovery Software,stellar-toolkit-for-iphone,Stellar Toolkit for iPhone is a comprehensive ...,"Gurgaon, Haryana",File Recovery
1,OpenText Voltage SmartCipher,Simplify unstructured data security with persi...,OpenText software applications manage content ...,https://www.opentext.com/,Confidentiality Software,opentext-voltage-smartcipher,SmartCipher simplifies unstructured data secur...,"Waterloo, ON",Encryption
2,IBM Maximo IT,Rapid expansion of information technology has ...,IBM offers a wide range of technology and cons...,https://www.ibm.com/,Service Desk Software,ibm-maximo-it,IBM Control Desk enables unified IT Service an...,"Armonk, NY","IT Service Management (ITSM) Tools"" ""Service Desk"
3,Panvalet,CA Panvalet is a library management system tha...,Broadcom Inc. (NASDAQ: AVGO) is a global techn...,https://www.broadcom.com/,DevOps Software,panvalet,CA Panvalet is a library management system tha...,"San Jose, CA",Version Control Software
4,OpenText ZENworks Patch Management,Micro Focus ZENworks Patch Management (formerl...,OpenText software applications manage content ...,https://www.opentext.com/,Vulnerability Management Software,opentext-zenworks-patch-management,Micro Focus ZENworks Patch Management (formerl...,"Waterloo, ON",Patch Management
...,...,...,...,...,...,...,...,...,...
75970,DentalPlus,DentalPlus is a practice management software t...,Admor is a well-known provider in the dental i...,https://dentalplus.co.uk/,Health Care Software,dentalplus,DentalPlus is a practice management software t...,UNKNOWN,Dental Practice Management
75971,DeMax,DeMax is a dealer management solution that all...,"Sundaram Infotech Solutions Limited, with its ...",https://www.sundaraminfotech.com/,Automotive Software,demax,DeMax is a dealer management solution that all...,"Tamilnadu, India",Car Dealer
75972,Demand Works Smoothie,Leading Software Tools for Sales & Operations ...,Netstock is a leader in demand and supply plan...,https://netstock.com/,Demand Planning Software,demand-works-smoothie,Leading Software Tools for Sales & Operations ...,"Boston, US",Demand Planning
75973,Design on Demand,get high-quality 3D models that are ready to m...,Zverse Inc is a technology company specializin...,https://zverse.com/,3D Design Software,design-on-demand,get high-quality 3D models that are ready to m...,UNKNOWN,3D Modeling


In [216]:
# unify categories 

# drop columns that we can't use for entity matching
dfs[0].drop(columns=["jobs", "companies", "companies_found_last_week", "technology_id", "slug", "category_slug", "parent_category_slug"], inplace=True)
dfs[1].drop(columns=["headquarters", "seller_description", "overview", "software_product_id"], inplace=True) # overview is virtually identical to description

In [217]:
def standardize_text(df, cols):
    for col in cols:
        df[col] = (
            df[col]
            .str.lower()
            .replace("[^a-z0-9]", " ", regex=True)  # replacing special characters with a space
            .replace(" +", " ", regex=True)         # removing consecutive spaces
            .str.strip()                            # removing leading and tailing spaces
        )

standardize_text(dfs[0], ["name", "description", "category", "parent_category"])
standardize_text(dfs[1], ["name", "description", "category", "parent_category"])

In [218]:
dfs[0]

,name,url,description,category,parent_category
0,activecampaign,http://www.activecampaign.com,recognized as the leader in the marketing and ...,marketing automation platforms,marketing
1,acuity scheduling,https://acuityscheduling.com,it is easy to use and user friendly scheduling...,appointments and scheduling,customer management
2,adobe illustrator,https://www.adobe.com/ru/products/illustrator....,the industry standard vector graphics app lets...,graphic design software,product and design
3,adobe photoshop,https://www.adobe.com/,it is the best in the world of graphic design ...,graphic design software,product and design
4,aftership,https://www.aftership.com,aftership provides shipment tracking api for o...,shipping and fulfillment,e commerce
...,...,...,...,...,...
32192,sketchfab,UNKNOWN,with a community of over one million creators ...,3d modeling,product and design
32193,pmweb,UNKNOWN,pmweb is the world s 1 all in one enterprise c...,construction management,productivity and operations
32194,murex,UNKNOWN,transform it infrastructure meet regulatory re...,investment and finance services,finance and accounting
32195,calypso,UNKNOWN,calypso is the leading provider of innovative ...,investment and finance services,finance and accounting


In [219]:
dfs[1]

,name,description,url,parent_category,category
0,stellar toolkit for iphone,stellar toolkit for iphone is a comprehensive ...,https://www.stellarinfo.com/,data recovery software,file recovery
1,opentext voltage smartcipher,simplify unstructured data security with persi...,https://www.opentext.com/,confidentiality software,encryption
2,ibm maximo it,rapid expansion of information technology has ...,https://www.ibm.com/,service desk software,it service management itsm tools service desk
3,panvalet,ca panvalet is a library management system tha...,https://www.broadcom.com/,devops software,version control software
4,opentext zenworks patch management,micro focus zenworks patch management formerly...,https://www.opentext.com/,vulnerability management software,patch management
...,...,...,...,...,...
75970,dentalplus,dentalplus is a practice management software t...,https://dentalplus.co.uk/,health care software,dental practice management
75971,demax,demax is a dealer management solution that all...,https://www.sundaraminfotech.com/,automotive software,car dealer
75972,demand works smoothie,leading software tools for sales operations pl...,https://netstock.com/,demand planning software,demand planning
75973,design on demand,get high quality 3d models that are ready to m...,https://zverse.com/,3d design software,3d modeling


In [220]:
# merge the datasets
df = pd.merge(dfs[0], dfs[1], how="outer")
assert df.shape[0] == dfs[0].shape[0] + dfs[1].shape[0], f"Outer join should have preserved all rows from both datasets, instead got {dfs[0].shape[0] + dfs[1].shape[0]} != {df.shape[0]}"

print("Total number of entities:", df.shape[0])

Total number of entities: 108172


## Product Deduplication

1. create blocks based on first word in name
2. compute the Jaro-Winkler similarity for names
    - == 1.0 -> match
    - <= 0.6 -> no match
3. compute cosine similarity for descriptions if Jaro-Winkler similarity > 0.6. 
    - > 0.8 -> match
    - else -> no match.

In [221]:
df

,name,url,description,category,parent_category
0,000webhost,http://www.000webhost.com,free web hosting with php mysql free website b...,domain name services,computer networks
1,000webhost,https://hostinger.com/,are your plants not thriving do you have weeds...,website builder wordpress hosting reseller hos...,website builder software
2,007names com,https://007names.com/,007names provides domains and hosting for webs...,wordpress hosting reseller hosting dedicated h...,web hosting providers
3,01supply,https://01supply.com/,01supply helps dropshipping retailers and supp...,e commerce tools,e commerce software
4,0chain,https://www.0chain.net/,0chain provides enterprises the ability to cus...,aws marketplace,aws marketplace software
...,...,...,...,...,...
108167,zywave client cloud solutions,https://www.zywave.com/,with zywave client cloud solutions you can del...,product suites,product suites
108168,zywave content cloud solutions,https://www.zywave.com/,with zywave content cloud solutions you can el...,product suites,product suites
108169,zywave hrconnection,UNKNOWN,unknown,payroll and benefits,hr
108170,zywave sales cloud solutions,https://www.zywave.com/,with zywave sales cloud solutions you can get ...,product suites,product suites


### Creating blocks

- I'm creating blocks only based on the first word in name - e.g. adobe illustrator and adobe photoshop

In [222]:
def standard_blocking(col_vals):

    blocks = defaultdict(list)
    for idx, key in enumerate(col_vals):
        first_keyword = str(key).split(sep=" ", maxsplit=1)[0]
        blocks[first_keyword].append(idx)

    return blocks

name_blocks = standard_blocking(df["name"])


In [223]:
name_blocks
max_len = 0
for key, val in name_blocks.items():
    max_len = max(max_len, len(val))
print(max_len) # we have a max len of 689 pairs 

def clean_blocks(blocks):
    cleaned_blocks = {}
    for key, val in blocks.items():
        if len(val) > 1:
            cleaned_blocks[key] = val
    return cleaned_blocks

blocks = clean_blocks(name_blocks)
blocks


689


{'000webhost': [0, 1],
 '1': [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
 '10': [22, 23, 24],
 '1000minds': [25, 26],
 '100hires': [30, 31],
 '101': [34, 35],
 '10duke': [44, 45],
 '10web': [48, 49],
 '11': [53, 54, 55],
 '123': [59, 60, 61, 62, 63, 64, 65, 66],
 '123signup': [83, 84],
 '12d': [88, 89],
 '12pay': [90, 91],
 '12return': [92, 93],
 '140proof': [97, 98],
 '15five': [101, 102],
 '15gifts': [103, 104],
 '17hats': [105, 106],
 '1c': [114, 115, 116],
 '1click': [117, 118, 119],
 '1crm': [121, 122],
 '1e': [128, 129],
 '1link': [141, 142, 143],
 '1ntelligence': [147, 148],
 '1password': [153, 154, 155],
 '1shoppingcart': [164, 165],
 '1st': [168, 169, 170, 171],
 '1stflip': [173, 174],
 '2': [188, 189, 190],
 '20': [191, 192, 193, 194, 195, 196],
 '24': [211, 212, 213],
 '24liveblog': [224, 225],
 '24sevenoffice': [231, 232],
 '2600hz': [235, 236],
 '2ask': [243, 244],
 '2bm': [247, 248],
 '2captcha': [249, 250],
 '2checkout': [251, 252, 253],
 '2degrees': [256,

In [224]:
sum_comparisons = 0
for key, val in blocks.items():
    sum_comparisons += len(val) * (len(val) - 1) // 2
sum_comparisons # this might be a lot of comparisons but let's see

1307429

In [225]:
from itertools import combinations

def create_pairs(blocks):
    pairs = []
    for _, idx in blocks.items():
        pairs.extend(combinations(idx, 2))
    return pairs

pairs = create_pairs(blocks)

### Text similarity matching
- Jaro-Winkler similarity on product names
- cosine similarity on product descriptions that have been tokenized 

In [228]:
def match_entities(df, pairs, jarowinkler_sim_threshold=0.6, cosine_sim_threshold=0.8):
    """
    Match entities based on Jaro-Winkler and cosine similarity.
    """

    duplicated_entity_ids = []
    jaro_count = 0
    cosine_count = 0

    vectorizer = CountVectorizer()
    vectorizer.fit([df["description"], df["category"]])

    for id1, id2 in tqdm(pairs):
        jarowinkler_sim = jellyfish.jaro_winkler_similarity(df.iloc[id1]["name"], df.iloc[id2]["name"])
        if jarowinkler_sim == 1.0: # automatically merge because these are obviously the same entities
            duplicated_entity_ids.append(id1)
            jaro_count += 1
            continue

        if jarowinkler_sim >= jarowinkler_sim_threshold: # these are similar enough to consider further
            if df.iloc[id1]["description"] != "unknown" and df.iloc[id2]["description"] != "unknown":
                col = "description"
            else:
                col = "category"

            vectors = vectorizer.transform([df.iloc[id1][col], df.iloc[id2][col]])
            cosine_sim = cosine_similarity(vectors[0], vectors[1])[0][0]
            print(cosine_sim)

            if cosine_sim >= cosine_sim_threshold:  # Threshold for merging based on category similarity
                duplicated_entity_ids.append(id1)
                cosine_count += 1
        
    print("Number of matching entities found by Jaro-Winkler similarity:", jaro_count)
    print("Number of matching entities found by cosine similarity:", cosine_count)

    return duplicated_entity_ids

duplicated_entity_ids = match_entities(df, pairs)
# 11603 722

AttributeError: 'Series' object has no attribute 'lower'

In [ ]:
% pip install sentence-transformers
from sentence_transformers import SentenceTransformer

def semantic_evaluation(pairs, model='all-MiniLM-L6-v2'):
    encoder = SentenceTransformer(model)
    
    def get_similarity(p1, p2):
        emb1 = encoder.encode([p1['name'] + ' ' + p1['description']])
        emb2 = encoder.encode([p2['name'] + ' ' + p2['description']])
        return cosine_similarity(emb1, emb2)[0][0]
    
    return [get_similarity(p1, p2) for p1, p2 in pairs]

similarities = semantic_evaluation(pairs)

### Merging the deduplicated entities

In [ ]:
master_catalog = df.loc[~df.index.isin(duplicated_entity_ids)]

In [ ]:
pd.to_csv(master_catalog, "master_catalog.csv", index=False)
master_catalog
# we started with 108172 and in the end got 96619 

,name,url,description,category,parent_category
1,000webhost,https://hostinger.com/,are your plants not thriving do you have weeds...,website builder wordpress hosting reseller hos...,website builder software
2,007names com,https://007names.com/,007names provides domains and hosting for webs...,wordpress hosting reseller hosting dedicated h...,web hosting providers
3,01supply,https://01supply.com/,01supply helps dropshipping retailers and supp...,e commerce tools,e commerce software
4,0chain,https://www.0chain.net/,0chain provides enterprises the ability to cus...,aws marketplace,aws marketplace software
5,0xpay,https://0xpay.app/,0xpay is a crypto and fiat processing solution...,cryptocurrency payment gateways,blockchain software
...,...,...,...,...,...
108167,zywave client cloud solutions,https://www.zywave.com/,with zywave client cloud solutions you can del...,product suites,product suites
108168,zywave content cloud solutions,https://www.zywave.com/,with zywave content cloud solutions you can el...,product suites,product suites
108169,zywave hrconnection,UNKNOWN,unknown,payroll and benefits,hr
108170,zywave sales cloud solutions,https://www.zywave.com/,with zywave sales cloud solutions you can get ...,product suites,product suites
